#**AI LAW LEVEL 1:  CHATBOTS**
---

##0.REFERENCE

https://claude.ai/share/28c0e2a9-7e47-4daa-964e-263513b25fe0

##1.CONTEXT


**Introduction: Why Legal AI Usage Requires Governance, Not Just Conversation**

If you've used ChatGPT on a website or Claude through a mobile app, you already understand the basic appeal of artificial intelligence chatbots. You type a question or request, the system responds in natural language, and within seconds you have an answer, draft, or explanation. It feels like having a knowledgeable assistant available instantly at any time. For casual uses—planning a vacation, drafting a birthday card, explaining a concept to your child—this informal approach works perfectly fine.

But legal practice operates under fundamentally different rules than casual conversation. When you use artificial intelligence as a lawyer, you're not just getting helpful suggestions for personal use. You're potentially creating work product that affects client rights, court proceedings, and professional liability. You're handling confidential information protected by attorney-client privilege. You're subject to ethics rules that require competence, supervision, and documentation. The stakes are dramatically higher, and the informal approach that works for casual use becomes professionally dangerous.

This notebook teaches you how to use artificial intelligence chatbots for legal drafting and client communication in a way that meets professional responsibility standards. It's not about avoiding these tools—they offer genuine efficiency gains for routine tasks. It's about using them appropriately, with proper controls, documentation, and human oversight. Think of it as the difference between having a casual conversation with a friend about legal issues versus providing formal legal advice to a client. The underlying communication might be similar, but the professional context requires completely different standards.

**What Makes Legal AI Usage Different**

When you use a consumer chatbot app, several things happen invisibly that create problems for lawyers. First, you typically paste your question or information directly into the interface without thinking about confidentiality. The service provider sees your input, stores it on their servers, and may use it to train future models. For vacation planning, this doesn't matter. For client information protected by attorney-client privilege, this is potentially a waiver of that privilege.

Second, consumer chatbots give you an answer with no audit trail. You see the response, maybe copy it somewhere, and move on. There's no record of what you asked, what parameters controlled the response, or what version of the model you used. For personal use, this is fine. For professional work that might be questioned months later by a client, opposing counsel, or a court, this lack of documentation is indefensible.

Third, consumer chatbots provide no systematic way to identify risks in their outputs. Artificial intelligence models can hallucinate facts, invent legal citations, or produce plausible-sounding but incorrect information. In casual use, you might catch obvious errors or they don't matter much. In legal work, an invented case citation in a court filing can result in sanctions, and incorrect factual statements in client communications can create malpractice liability.

Fourth, consumer interfaces encourage treating artificial intelligence as an authority rather than a tool. The conversational format makes outputs feel like expert advice. But these systems don't "know" things the way humans do—they predict probable text patterns based on training data. They can be confidently wrong. Lawyers need to maintain appropriate professional distance and skepticism, which the casual chat interface doesn't encourage.

**What This Notebook Does Differently**

This notebook implements what we call Level One governance controls—the minimum standards for safe legal use of chatbot artificial intelligence. Every interaction with the model happens through a wrapper that implements multiple safeguards. Before sending any text to the model, the system runs it through redaction to remove personally identifiable information. This reduces but doesn't eliminate confidentiality risks—it's a harm-reduction approach acknowledging that perfect protection isn't possible with external services.

Every interaction gets logged with timestamps, cryptographic fingerprints, and redacted content previews. These logs create an audit trail showing what you asked, what the model returned, and what risks were identified. If you ever need to explain your artificial intelligence usage—to a client questioning a bill, to opposing counsel in discovery, to a disciplinary board investigating a complaint—you have comprehensive documentation showing you used the technology responsibly.

The system enforces structured output from the model. Rather than accepting free-form text that might omit critical information, every response must include specific fields: what facts were provided, what assumptions the model made, what questions remain unanswered, what risks were identified, what verification is needed, and the actual draft output. This structure forces both the model and you as the user to think systematically about completeness and limitations.

Automatic risk detection scans every output for warning signs. If the model produces something that looks like a legal citation—a case name, a statute number—the system flags potential hallucination, since the model wasn't asked to conduct legal research and may be inventing authorities. If the model identifies no open questions despite receiving limited facts, the system flags potential overconfidence. These automated checks catch common failure modes without requiring you to manually inspect every detail.

The notebook generates governance artifacts for every session: a manifest documenting what system and parameters you used, a complete log of all prompts and responses, a risk register tracking all identified concerns, and human-readable deliverables for each task. At the end, everything packages into a zip file you can download and retain with your file, satisfying documentation requirements for quality control and file retention.

**Why Chapter One Matters**

This is Chapter One of a progression through increasingly sophisticated artificial intelligence uses in legal practice. We start here—with simple drafting based on provided facts—because this represents the lowest-risk, highest-value use case for most lawyers today. You're not asking the artificial intelligence to conduct legal research, analyze complex legal issues, or make strategic judgments. You're asking it to help draft routine communications based on facts you provide. The model serves as a writing assistant, not a legal analyst.

But even this simple use case requires the governance framework you'll learn here. Every control implemented in this notebook—redaction, structured output, logging, risk detection, human review—addresses a real professional responsibility risk that has caused actual problems for lawyers using artificial intelligence without proper safeguards. Bar associations have issued sanctions for invented citations, malpractice claims have been filed over incorrect artificial intelligence-generated advice, and privilege waivers have occurred through careless data handling.

The habits you develop in Chapter One—minimizing sensitive inputs, maintaining logs, flagging risks, requiring human review—scale up to more advanced uses. When you move to Chapter Two covering legal research assistance, or Chapter Three covering more autonomous workflows, you'll build on this foundation rather than starting from scratch. The governance mindset matters more than any specific technical control, and this chapter establishes that mindset through hands-on practice.

You'll work through four realistic scenarios spanning different practice areas: a criminal bail letter, a regulatory comment outline, an international contract email to a client, and an academic policy on student artificial intelligence use. Each scenario demonstrates both the capability and the limitations of current artificial intelligence tools. You'll see what works well—routine drafting from clear facts—and what doesn't—inventing legal authorities or making judgment calls. This calibration of appropriate expectations is itself a professional competence requirement.

By the end of this notebook, you'll have created a complete audit trail for a governed artificial intelligence session. More importantly, you'll understand why each control exists, what risks it addresses, and how to adapt the approach to your specific practice needs. This isn't paint-by-numbers compliance—it's building genuine professional competence with an increasingly important tool category.


##2.LIBRARIES AND ENVIRONMENT

In [2]:
# Cell 2: Install + Imports

print("Installing dependencies...")
!pip install -q anthropic

print("\nImporting libraries...")
import json
import os
import re
import hashlib
import platform
from datetime import datetime
from pathlib import Path
from textwrap import dedent
import subprocess

# Create base run directory with timestamp
timestamp = datetime.now().strftime("%Y%m%d_%H%M%S")
RUN_DIR = Path(f"/content/ai_law_ch1_runs/run_{timestamp}")
RUN_DIR.mkdir(parents=True, exist_ok=True)

# Create subdirectories
DELIVERABLES_DIR = RUN_DIR / "deliverables"
DELIVERABLES_DIR.mkdir(exist_ok=True)

print(f"\n✅ Run directory created: {RUN_DIR}")
print(f"✅ Deliverables directory: {DELIVERABLES_DIR}")

Installing dependencies...
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 388.2/388.2 kB 8.6 MB/s eta 0:00:00

Importing libraries...

✅ Run directory created: /content/ai_law_ch1_runs/run_20260107_131424
✅ Deliverables directory: /content/ai_law_ch1_runs/run_20260107_131424/deliverables


##3.ANTRHOPIC INITIALIZATION

###3.1.OVERVIEW

**Section 3: API Key and Anthropic Client Initialization**

This section connects your notebook to Claude by setting up authentication. Think of it like logging into a secure service—you need valid credentials before you can use the AI model.

The section uses Google Colab's built-in secrets manager to store your API key safely. Instead of typing your key directly into the code where others might see it, you add it through Colab's interface by clicking the key icon in the sidebar. This approach protects your credentials from accidental exposure when sharing notebooks.

When the code runs, it attempts to retrieve your API key from the secrets storage. If successful, you see a confirmation message. If the key is missing, the notebook doesn't just crash—it provides clear step-by-step instructions showing exactly how to add the secret. This user-friendly error handling is especially important for lawyers who may be less familiar with programming environments.

Once authenticated, the code initializes the Anthropic client object, which acts as your connection to Claude throughout the notebook. The code also specifies exactly which version of Claude to use by setting the model string. This precision matters for professional responsibility because different model versions can produce different outputs. By locking in a specific version, you ensure reproducibility—running the notebook again later will use the same AI model, not whatever newest version exists.

The confirmation messages that appear after successful setup serve multiple purposes. First, they provide immediate feedback that everything worked correctly. Second, they display the exact model version being used, which becomes part of your documentation trail. If you ever need to explain to a client, court, or ethics board which AI system you used, this information provides that answer.

From a governance perspective, this section establishes the foundation of your audit trail. The model version recorded here will be written to your run manifest in the next section. This creates a clear record of exactly which AI system generated your outputs.

The separation between authentication in this section and logging in later sections follows good design principles. Each cell has one clear responsibility. If your organization uses different API key management in the future, you can modify just this section without affecting the rest of the notebook's functionality.

This setup also acknowledges that AI services are external tools requiring credentials, not magic boxes that just work. By making authentication explicit and visible, the notebook reinforces that you're using a commercial service with clear boundaries and controls. This transparency is important for maintaining appropriate professional distance from the technology—you're using a tool, not delegating professional judgment.

###3.2.CODE AND IMPLEMENTATION

In [3]:
# Cell 3: API Key + Anthropic Client Initialization

import anthropic

# Get API key from Colab userdata (secrets)
from google.colab import userdata

try:
    ANTHROPIC_API_KEY = userdata.get('ANTHROPIC_API_KEY')
    os.environ["ANTHROPIC_API_KEY"] = ANTHROPIC_API_KEY
    api_key_loaded = True
    print("✅ API key loaded: yes")
except Exception as e:
    api_key_loaded = False
    print("❌ API key loaded: no")
    print("\n⚠️ ERROR: Anthropic API key not found!")
    print("\nTo fix this:")
    print("1. Click the 🔑 key icon in the left sidebar")
    print("2. Add a new secret named: ANTHROPIC_API_KEY")
    print("3. Paste your Anthropic API key as the value")
    print("4. Enable 'Notebook access' toggle")
    print("5. Re-run this cell")
    raise ValueError("Missing ANTHROPIC_API_KEY in Colab secrets")

# Initialize Anthropic client
client = anthropic.Anthropic(api_key=os.environ["ANTHROPIC_API_KEY"])
MODEL = "claude-sonnet-4-5-20250929"

print(f"✅ Model: {MODEL}")
print(f"✅ Anthropic client initialized")

✅ API key loaded: yes
✅ Model: claude-sonnet-4-5-20250929
✅ Anthropic client initialized


##4.GOVERNANCE MANIFEST AND LOGGING UTILITIES

###4.1.OVERVIEW


**Section 4: Governance - Manifest and Logging Utilities**

This section builds the infrastructure for creating a complete audit trail of your AI usage. Every function here exists to document what you did, when you did it, and under what conditions—core requirements for professional responsibility.

The utility functions create the basic building blocks for record-keeping. One function generates timestamps in a standard format that works across different time zones and computer systems. Another writes data to files in a structured format that both humans and computers can read. A third creates cryptographic fingerprints of text, allowing you to verify later that logged content hasn't been altered without storing full copies of potentially confidential information.

The environment fingerprint function captures technical details about your computing setup—Python version, operating system, installed software packages. This might seem excessive, but it serves a critical purpose: reproducibility. If you need to recreate these exact results months later, perhaps because a client questions an output or opposing counsel challenges your methods, you need to know exactly what software environment produced the original results. Different software versions can produce subtly different outputs, so documenting your environment is like noting which edition of a legal treatise you cited.

The run manifest acts as the master record for this session. It captures who authored the notebook, which AI model was used, what parameters controlled the model's behavior, when everything ran, and what the notebook's purpose was. The temperature setting recorded here controls how creative versus consistent the AI's outputs will be—lower numbers mean more consistent, predictable responses. The disclaimer embedded in the manifest establishes upfront that all outputs are drafts requiring human lawyer review, not final work product.

Three separate artifact files work together as your audit system. The manifest answers what system you used and when. The prompts log will record what inputs you sent and what outputs you received. The risk log will track concerns identified during the process. Together, these files provide comprehensive documentation that demonstrates professional competence and appropriate supervision of AI tools.

From an ethics standpoint, this section implements the competence requirement that many state bars emphasize in their AI guidance. By documenting the AI's parameters and maintaining detailed logs, you're showing that you understand the tool you're using. You're not treating it as a black box—you know what model version, what settings, what inputs, and what outputs occurred.

The files are created immediately, not saved only at the end. This design choice means that even if your notebook crashes midway through, you still have partial records showing what was attempted. This fail-safe approach acknowledges that real-world usage isn't always perfect, and having some documentation is better than none.


###4.2.CODE AND IMPLEMENTATION

In [4]:
# Cell 4: Governance: Manifest + Logging Utilities

def now_iso():
    """Return current timestamp in ISO format"""
    return datetime.now().isoformat()

def write_json(path, obj):
    """Write object to JSON file"""
    with open(path, 'w') as f:
        json.dump(obj, f, indent=2)
    return path

def append_jsonl(path, record):
    """Append record to JSONL file"""
    with open(path, 'a') as f:
        f.write(json.dumps(record) + '\n')
    return path

def sha256_text(s):
    """Return SHA-256 hash of text"""
    return hashlib.sha256(s.encode()).hexdigest()

def get_env_fingerprint():
    """Capture environment details for reproducibility"""
    # Get pip freeze output
    try:
        pip_output = subprocess.check_output(['pip', 'freeze'], text=True)
        pip_file = RUN_DIR / "pip_freeze.txt"
        with open(pip_file, 'w') as f:
            f.write(pip_output)
    except:
        pip_output = "Unable to capture pip freeze"

    return {
        "python_version": platform.python_version(),
        "platform": platform.platform(),
        "timestamp": now_iso(),
        "pip_freeze_file": str(pip_file) if 'pip_file' in locals() else None
    }

# Create run manifest
manifest = {
    "run_id": f"ch1_run_{timestamp}",
    "timestamp": now_iso(),
    "author": "Alejandro Reynoso, Chief Scientist DEFI CAPITAL RESEARCH",
    "model": MODEL,
    "parameters": {
        "temperature": 0.2,
        "max_tokens": 1200
    },
    "environment": get_env_fingerprint(),
    "notebook_purpose": "Chapter 1 - Level 1 Chatbots: Drafting + Client Communication with Governance Controls",
    "disclaimer": "NOT LEGAL ADVICE. Human lawyer review required. All outputs are drafts only."
}

manifest_path = RUN_DIR / "run_manifest.json"
write_json(manifest_path, manifest)
print(f"✅ Manifest created: {manifest_path}")

# Initialize empty prompt log (JSONL format)
prompts_log_path = RUN_DIR / "prompts_log.jsonl"
prompts_log_path.touch()
print(f"✅ Prompts log initialized: {prompts_log_path}")

# Initialize risk log
risk_log = {
    "run_id": manifest["run_id"],
    "timestamp": now_iso(),
    "risks": []
}
risk_log_path = RUN_DIR / "risk_log.json"
write_json(risk_log_path, risk_log)
print(f"✅ Risk log initialized: {risk_log_path}")

print(f"\n📁 All governance artifacts ready in: {RUN_DIR}")

✅ Manifest created: /content/ai_law_ch1_runs/run_20260107_131424/run_manifest.json
✅ Prompts log initialized: /content/ai_law_ch1_runs/run_20260107_131424/prompts_log.jsonl
✅ Risk log initialized: /content/ai_law_ch1_runs/run_20260107_131424/risk_log.json

📁 All governance artifacts ready in: /content/ai_law_ch1_runs/run_20260107_131424


##5.CONDIFENTIALITY UTILITIES

###5.1.OVERVIEW

**Section 5: Confidentiality Utilities - Redaction and Minimal Input Builder**

This section addresses one of the most serious risks in legal AI usage: accidentally sending confidential client information to external services. When you transmit data to an AI provider, you may waive attorney-client privilege and work-product protection. This section provides technical controls to reduce that risk, though it cannot eliminate it entirely.

The redaction function automatically scans text for common types of sensitive information and masks them before sending anything to the AI. It looks for patterns like email addresses, phone numbers, Social Security numbers, street addresses, and names with titles. When it finds these patterns, it replaces them with placeholder text and creates a list of what was removed. This gives you transparency into the sanitization process.

The function acknowledges its limitations openly. The pattern-matching approach is imperfect by design. For example, the name detection only catches names with titles like Mister or Doctor, so it will miss many real names. The address detection only recognizes common street suffixes, so unusual or international addresses may slip through. These limitations are inherent to automated pattern-matching, which is why the code includes prominent warnings that human review remains essential.

The minimal input builder applies an important principle: only send AI systems the information they absolutely need for the specific task. By tracking what was removed and comparing original versus redacted length, it creates a record of your sanitization process. This documentation matters for professional responsibility—if ever questioned, you can show you took concrete steps to minimize data exposure.

The demonstration using fake data serves two purposes. First, it shows users exactly how the function works with concrete examples. Second, it models safe behavior—always test with fake data before using real client information. This pedagogical choice reinforces good habits.

From a professional responsibility perspective, this section implements the reasonable measures standard for protecting confidentiality. If your AI usage were ever challenged, you could point to this code as evidence that you implemented technical controls before transmission. That's not absolute protection, but it demonstrates taking confidentiality seriously rather than treating it as an afterthought.

The warning messages throughout this section create legal notice. Users cannot later claim they didn't understand the risks. The notebook explicitly tells them that privilege may not be retained, automated redaction isn't foolproof, and manual review is required.

For organizations with strict data governance policies, this section provides a foundation that can be extended. You can add more patterns, integrate with enterprise redaction tools, or replace the entire approach while maintaining the same interface with the rest of the notebook.

###5.2.CODE AND IMPLEMENTATION

In [6]:
# Cell 5: Confidentiality Utilities: Redaction + Minimal-Input Builder

def redact(text):
    """
    Redact sensitive information from text.
    Returns: (redacted_text, removed_fields_list)

    WARNING: This is a best-effort heuristic. Not foolproof.
    Always manually review before sending to external services.
    """
    removed = []
    result = text

    # Email addresses
    email_pattern = r'\b[A-Za-z0-9._%+-]+@[A-Za-z0-9.-]+\.[A-Z|a-z]{2,}\b'
    emails_found = re.findall(email_pattern, result)
    if emails_found:
        result = re.sub(email_pattern, '[EMAIL_REDACTED]', result)
        removed.append(f"Emails: {len(emails_found)} found")

    # Phone numbers (US format, basic patterns)
    phone_pattern = r'\b(?:\+?1[-.]?)?\(?([0-9]{3})\)?[-.]?([0-9]{3})[-.]?([0-9]{4})\b'
    phones_found = re.findall(phone_pattern, result)
    if phones_found:
        result = re.sub(phone_pattern, '[PHONE_REDACTED]', result)
        removed.append(f"Phone numbers: {len(phones_found)} found")

    # SSN patterns (XXX-XX-XXXX)
    ssn_pattern = r'\b\d{3}-\d{2}-\d{4}\b'
    ssns_found = re.findall(ssn_pattern, result)
    if ssns_found:
        result = re.sub(ssn_pattern, '[SSN_REDACTED]', result)
        removed.append(f"SSNs: {len(ssns_found)} found")

    # Street addresses (simplified: number + street name pattern)
    # This is imperfect but catches common patterns
    address_pattern = r'\b\d{1,6}\s+[A-Z][a-z]+\s+(Street|St|Avenue|Ave|Road|Rd|Boulevard|Blvd|Lane|Ln|Drive|Dr|Court|Ct)\b'
    addresses_found = re.findall(address_pattern, result, re.IGNORECASE)
    if addresses_found:
        result = re.sub(address_pattern, '[ADDRESS_REDACTED]', result, flags=re.IGNORECASE)
        removed.append(f"Street addresses: {len(addresses_found)} found")

    # Full names (very simple heuristic: Title + FirstName + LastName)
    # WARNING: This will have false positives/negatives
    name_pattern = r'\b(Mr\.|Mrs\.|Ms\.|Dr\.|Prof\.)\s+[A-Z][a-z]+\s+[A-Z][a-z]+\b'
    names_found = re.findall(name_pattern, result)
    if names_found:
        result = re.sub(name_pattern, '[NAME_REDACTED]', result)
        removed.append(f"Names (with titles): {len(names_found)} found")

    return result, removed

def build_minimal_input(scenario_text):
    """
    Convert user scenario into minimal necessary facts.
    Returns: dict with sanitized_facts and removed_fields
    """
    redacted_text, removed = redact(scenario_text)

    # Extract bullet points if present, otherwise treat as single fact
    facts = []
    for line in redacted_text.split('\n'):
        line = line.strip()
        if line:
            facts.append(line)

    return {
        "sanitized_facts": facts,
        "removed_fields": removed,
        "original_length": len(scenario_text),
        "redacted_length": len(redacted_text)
    }

# Demo with fake data
print("=== REDACTION DEMO ===\n")
demo_text = """
Client John Smith contacted us at john.smith@example.com.
His phone is 555-123-4567 and SSN is 123-45-6789.
He lives at 123 Main Street in the city.
Mr. John Smith needs assistance with a contract.
"""

print("BEFORE REDACTION:")
print(demo_text)

redacted_demo, removed_demo = redact(demo_text)
print("\nAFTER REDACTION:")
print(redacted_demo)

print("\nREMOVED FIELDS:")
for item in removed_demo:
    print(f"  - {item}")

print("\n⚠️ IMPORTANT: This is best-effort only. Always manually review sensitive data.")

=== REDACTION DEMO ===

BEFORE REDACTION:

Client John Smith contacted us at john.smith@example.com.
His phone is 555-123-4567 and SSN is 123-45-6789.
He lives at 123 Main Street in the city.
Mr. John Smith needs assistance with a contract.


AFTER REDACTION:

Client John Smith contacted us at [EMAIL_REDACTED].
His phone is [PHONE_REDACTED] and SSN is [SSN_REDACTED].
He lives at [ADDRESS_REDACTED] in the city.
[NAME_REDACTED] needs assistance with a contract.


REMOVED FIELDS:
  - Emails: 1 found
  - Phone numbers: 1 found
  - SSNs: 1 found
  - Street addresses: 1 found
  - Names (with titles): 1 found

⚠️ IMPORTANT: This is best-effort only. Always manually review sensitive data.


##6.LLM CODE WRAPPER

###6.1.OVERVIEW

**Section 6: LLM Call Wrapper with Strict JSON Parsing and Risk Flags**

This section is the technical heart of the governance system. It wraps every call to Claude with multiple layers of control: input sanitization, structured output enforcement, retry logic, automatic risk detection, and comprehensive logging. This transforms a simple API call into a professionally defensible process.

The JSON extraction function solves a practical problem: artificial intelligence models don't always follow instructions perfectly. Even when explicitly told to return only structured data with no extra text, Claude sometimes adds explanatory preambles or wraps output in formatting marks. This function implements four different strategies to find valid structured data within messy responses. It tries parsing the whole response first, then looks for content between brackets, handles formatting blocks, and finally uses sophisticated bracket-balancing to extract complete data structures. This belt-and-suspenders approach ensures robustness in real-world usage.

The main wrapper function builds an extremely explicit instruction set for Claude. Rather than assuming the model will understand "return structured data," it repeats the instruction multiple times in different ways, shows the exact structure to copy, lists what not to do, and provides a final reminder. This repetitive approach compensates for the fact that language models sometimes miss instructions buried in long prompts.

The retry logic implements three attempts with progressively stricter instructions and lower creativity settings. First attempt uses standard instructions with minimal creativity. If that fails, the second attempt adds even more explicit constraints and eliminates creativity entirely. Third attempt repeats this approach. If all three fail, the system returns a valid error structure rather than crashing, ensuring the notebook continues running even when individual tasks encounter problems.

Automatic risk detection analyzes outputs for warning signs. If the response contains no questions about missing information, the system flags potential incompleteness. If the draft contains patterns that look like legal citations—case names, statute numbers, court references—the system flags potential hallucination, since the model might be inventing authorities that don't exist. These automated checks catch common failure modes without requiring manual inspection of every output.

Comprehensive logging records every interaction. Each API call generates a log entry with timestamps, task identifiers, cryptographic fingerprints of inputs and outputs, and success indicators. The prompts log uses a streaming format that allows incremental writes, so even if the notebook crashes, you have records of everything that happened before the crash. The risk log accumulates all flagged concerns across all tasks, creating a master list of issues requiring human review.

From a professional responsibility standpoint, this section implements appropriate supervision of AI tools. You're not blindly accepting whatever the model produces—you're validating structure, detecting risks, maintaining records, and building in redundancy. If questioned about your AI usage, you can demonstrate that you implemented multiple safeguards rather than treating the technology as infallible.

###6.2.CODE AND IMPLEMENTATION

In [13]:
# Cell 6: LLM Call Wrapper with ROBUST JSON Extraction + Risk Flags

def extract_json_from_text(text):
    """
    Robustly extract JSON from text that may contain extra content.
    Tries multiple strategies to find and extract valid JSON.
    """
    # Strategy 1: Try the whole text as-is
    try:
        return json.loads(text.strip())
    except json.JSONDecodeError:
        pass

    # Strategy 2: Look for JSON between curly braces (most common pattern)
    # Find the first { and last }
    first_brace = text.find('{')
    last_brace = text.rfind('}')

    if first_brace != -1 and last_brace != -1 and last_brace > first_brace:
        try:
            json_str = text[first_brace:last_brace + 1]
            return json.loads(json_str)
        except json.JSONDecodeError:
            pass

    # Strategy 3: Remove markdown code blocks
    # Pattern: ```json ... ``` or ``` ... ```
    import re
    code_block_pattern = r'```(?:json)?\s*(\{.*?\})\s*```'
    matches = re.findall(code_block_pattern, text, re.DOTALL)
    if matches:
        for match in matches:
            try:
                return json.loads(match)
            except json.JSONDecodeError:
                continue

    # Strategy 4: Try to find largest JSON-like structure
    # Look for balanced braces
    stack = []
    start_idx = -1

    for i, char in enumerate(text):
        if char == '{':
            if not stack:
                start_idx = i
            stack.append(char)
        elif char == '}':
            if stack:
                stack.pop()
                if not stack and start_idx != -1:
                    # Found a complete JSON object
                    try:
                        return json.loads(text[start_idx:i+1])
                    except json.JSONDecodeError:
                        continue

    # All strategies failed
    return None


def call_claude(task_name, user_prompt, facts):
    """
    Call Claude with governance controls and ROBUST JSON handling.

    This version is designed to handle Claude's tendency to add explanatory
    text around JSON responses.
    """

    # Build system prompt with VERY explicit JSON-only instruction
    system_prompt = dedent("""
    You are a legal drafting assistant. You MUST respond with ONLY a JSON object.

    CRITICAL INSTRUCTION: Your entire response must be ONLY the JSON object below.
    Do not write "Here is the JSON" or "```json" or any other text.
    Start your response with { and end with }. Nothing before, nothing after.

    Required JSON structure (copy this exactly and fill in values):
    {
      "task": "brief description of what you're drafting",
      "facts_provided": ["fact 1", "fact 2"],
      "assumptions": ["assumption 1", "assumption 2"],
      "open_questions": ["question 1", "question 2"],
      "risks": [
        {"type": "missing_facts", "severity": "medium", "note": "explanation of risk"}
      ],
      "draft_output": "Your draft text here. Must include: This is a draft only. Not legal advice. Human lawyer review required.",
      "verification_status": "Not verified",
      "questions_to_verify": ["any claims that need verification"]
    }

    Rules:
    - Valid risk types: confidentiality, privilege, hallucination, missing_facts, unauthorized_practice, tone, other
    - Valid severity: low, medium, high
    - NEVER invent citations, case names, or legal authorities
    - If information is missing, list it in open_questions
    - Always include disclaimer in draft_output
    - verification_status must always be "Not verified"

    REMEMBER: Response = JSON only. No explanation. No markdown. Just the JSON object.
    """).strip()

    # Combine facts and prompt
    facts_text = "\n".join(f"- {fact}" for fact in facts)
    full_prompt = f"{user_prompt}\n\nFACTS PROVIDED:\n{facts_text}"

    # Redact the prompt
    full_prompt_redacted, _ = redact(full_prompt)

    max_retries = 3
    result = None

    for attempt in range(max_retries):
        try:
            # Adjust temperature based on attempt
            temp = 0.0 if attempt > 0 else 0.1

            # Call Claude API
            response = client.messages.create(
                model=MODEL,
                max_tokens=1500,  # Increased slightly
                temperature=temp,
                system=system_prompt,
                messages=[
                    {"role": "user", "content": full_prompt_redacted}
                ]
            )

            response_text = response.content[0].text

            # Try to extract JSON using robust method
            result = extract_json_from_text(response_text)

            if result is not None:
                # Success! Validate it has required keys
                required_keys = ["task", "facts_provided", "assumptions", "open_questions",
                               "risks", "draft_output", "verification_status", "questions_to_verify"]

                missing_keys = [k for k in required_keys if k not in result]

                if missing_keys:
                    print(f"  ⚠️ JSON missing keys: {missing_keys} (adding defaults)")
                    # Add defaults for missing keys
                    defaults = {
                        "task": task_name,
                        "facts_provided": facts,
                        "assumptions": [],
                        "open_questions": ["Response incomplete - missing information"],
                        "risks": [],
                        "draft_output": "Incomplete response received. Human review required.",
                        "verification_status": "Not verified",
                        "questions_to_verify": []
                    }
                    for key in missing_keys:
                        result[key] = defaults[key]

                # Success - break out of retry loop
                print(f"  ✅ Valid JSON extracted (attempt {attempt + 1})")
                break

            else:
                # Could not extract JSON
                if attempt < max_retries - 1:
                    print(f"  ⚠️ JSON extraction failed (attempt {attempt + 1}). Retrying with stricter prompt...")
                    # Modify system prompt for next attempt
                    system_prompt = "Output ONLY a valid JSON object. No text before or after. Start with { and end with }.\n\n" + system_prompt
                else:
                    print(f"  ❌ Could not extract JSON after {max_retries} attempts")

        except Exception as e:
            if attempt < max_retries - 1:
                print(f"  ⚠️ API error (attempt {attempt + 1}): {str(e)[:100]}")
            else:
                print(f"  ❌ API call failed after {max_retries} attempts: {str(e)[:100]}")

    # If we still don't have a result, create error structure
    if result is None:
        print(f"  ❌ Could not get valid JSON from Claude for {task_name}")
        result = {
            "task": task_name,
            "error": "Could not extract valid JSON from model response",
            "facts_provided": facts,
            "assumptions": [],
            "open_questions": ["Model did not return valid structured output"],
            "risks": [
                {
                    "type": "other",
                    "severity": "high",
                    "note": "System error: Could not parse model response as JSON"
                }
            ],
            "draft_output": "ERROR: Could not generate valid response. Please try again or contact support.",
            "verification_status": "Not verified",
            "questions_to_verify": []
        }

    # Auto-compute additional risk flags
    auto_risks = []

    # Check for missing open_questions
    if not result.get("open_questions") or len(result.get("open_questions", [])) == 0:
        auto_risks.append({
            "type": "missing_facts",
            "severity": "medium",
            "note": "No open questions identified, but facts may be incomplete"
        })

    # Check for citation-like patterns
    draft = result.get("draft_output", "")
    citation_patterns = [
        r'\d+\s+[A-Z][a-z]+\.?\s+\d+',  # "123 F.3d 456"
        r'§\s*\d+',  # "§ 123"
        r'\b[A-Z][a-z]+\s+v\.\s+[A-Z][a-z]+\b'  # "Smith v. Jones"
    ]

    for pattern in citation_patterns:
        if re.search(pattern, draft):
            auto_risks.append({
                "type": "hallucination",
                "severity": "high",
                "note": "Draft contains citation-like patterns. Verify all legal references before use."
            })
            break

    # Add auto-detected risks
    if auto_risks:
        existing_risks = result.get("risks", [])
        result["risks"] = existing_risks + auto_risks

    # Log to prompts_log.jsonl
    try:
        log_entry = {
            "timestamp": now_iso(),
            "task_name": task_name,
            "prompt_hash": sha256_text(full_prompt_redacted),
            "prompt_preview": full_prompt_redacted[:300] + "..." if len(full_prompt_redacted) > 300 else full_prompt_redacted,
            "response_hash": sha256_text(str(result)),
            "model": MODEL,
            "attempts": attempt + 1 if 'attempt' in locals() else 1,
            "success": "error" not in result
        }
        append_jsonl(prompts_log_path, log_entry)
    except Exception as e:
        print(f"  ⚠️ Warning: Could not log to prompts_log.jsonl: {e}")

    # Add risks to risk_log
    try:
        for risk in result.get("risks", []):
            risk_entry = {
                "timestamp": now_iso(),
                "task_name": task_name,
                "risk_type": risk.get("type", "other"),
                "severity": risk.get("severity", "low"),
                "note": risk.get("note", "")
            }
            risk_log["risks"].append(risk_entry)

        # Update risk log file
        write_json(risk_log_path, risk_log)
    except Exception as e:
        print(f"  ⚠️ Warning: Could not update risk_log.json: {e}")

    return result


# Enhanced smoke test
print("=" * 60)
print("SMOKE TEST - Testing JSON Extraction")
print("=" * 60)

test_result = call_claude(
    "smoke_test",
    "Draft a one-sentence client email acknowledging receipt of documents.",
    ["Client sent contract for review yesterday"]
)

if "error" not in test_result:
    print("\n✅ Wrapper ready and functional!")
    print(f"   Task: {test_result.get('task', 'N/A')}")
    print(f"   Risks: {len(test_result.get('risks', []))} identified")
    print(f"   Draft preview: {test_result.get('draft_output', '')[:80]}...")
else:
    print("\n⚠️ Smoke test completed with errors")
    print(f"   Error: {test_result.get('error', 'Unknown')}")
    print("   The system will continue but may have issues.")

print("=" * 60)

SMOKE TEST - Testing JSON Extraction
  ✅ Valid JSON extracted (attempt 1)

✅ Wrapper ready and functional!
   Task: Draft one-sentence client email acknowledging receipt of documents
   Risks: 2 identified
   Draft preview: Thank you for sending the contract yesterday; I have received it and will review...


##7.MINI CASE BUILDERS

###7.1.OVERVIEW

**Section 7: Mini-Case Builders - Four Recurring Scenarios**

This section defines four realistic legal scenarios that demonstrate how artificial intelligence can assist with routine drafting tasks under appropriate governance controls. Each scenario represents a different practice area and communication type, showing the breadth of potential applications while maintaining consistent safety standards.

The criminal bail case provides concrete facts about a defendant charged with a non-violent offense. Rather than vague details, it specifies employment history, family ties, and residence information—the kinds of facts that actually matter in bail decisions. The prompt asks for a brief letter with clear word limits and specific content requirements. This bounded approach prevents the model from wandering into speculation or producing excessive output.

The regulatory comment case involves a bank responding to proposed federal rules. Again, the facts are specific: bank size, regulation details, implementation timeline, and estimated costs. The prompt requests an outline rather than a complete letter, acknowledging that complex regulatory work requires significant human expertise beyond what artificial intelligence can provide. The structure guides the model toward practical concerns—implementation challenges, costs, timeline requests—rather than legal analysis.

The international contract case addresses cross-border commercial transactions. The facts establish context: which countries, what type of business, what the vendor proposed. The prompt asks for a client-friendly email explaining complex concepts in plain language. This scenario recognizes that lawyers often serve as translators between legal technicalities and business realities. The model helps with clear explanation, not legal advice about which option to choose.

The teaching policy case demonstrates that governance frameworks apply beyond client work. Law professors need clear policies about student use of artificial intelligence tools. The facts specify the course, institution, and instructor preferences. The prompt requests rules plus rationale—not just telling students what to do, but explaining why certain restrictions matter pedagogically.

Each case follows the same structural pattern: concrete facts with specific details, clear task description with word limits, content requirements in bullet format, tone guidance, and a reminder to include the required disclaimer. This consistency makes the cases predictable for the model while allowing variation in substance.

The prompts deliberately avoid open-ended meta-questions like "identify what information you need." Such questions invite prose explanations rather than structured output. Instead, the structured data format includes a field for open questions, allowing the model to flag missing information naturally within the required schema.

From a pedagogical perspective, these four scenarios show lawyers that governance doesn't mean avoiding artificial intelligence—it means using it appropriately. Simple drafting tasks based on provided facts fall within current capabilities. Complex analysis, legal research requiring citations, or judgment calls about strategy remain human responsibilities. By demonstrating both appropriate and bounded use cases, the notebook teaches sound professional judgment about when and how to employ these tools.

###7.2.CODE AND IMPLEMENTATION

In [16]:
# Cell 7: Mini-Case Builders (4 recurring scenarios) - SIMPLIFIED

def case_criminal_bail():
    """Criminal: Bail letter / mitigation narrative draft"""
    return {
        "task_name": "criminal_bail_letter",
        "facts": [
            "Defendant: John Doe (name redacted for exercise)",
            "Charge: Non-violent property offense (theft under $500)",
            "Criminal history: None",
            "Employment: Full-time warehouse worker for 3 years at ABC Company",
            "Residence: Lives with spouse and two children in the county",
            "Family ties: Elderly mother lives nearby, defendant provides care"
        ],
        "prompt": """Task: Draft a brief bail letter to the court (250 words maximum).

Content to include:
- Request for release pending trial
- Emphasize community ties (employment, family)
- Note lack of flight risk
- Professional, respectful tone

Output format: A concise letter suitable for court filing.

Include the required disclaimer at the end."""
    }

def case_regulatory_comment():
    """Regulatory/Administrative: Draft comment letter outline to agency"""
    return {
        "task_name": "regulatory_comment_letter",
        "facts": [
            "Client: First Community Bank (regional bank, $2B in assets)",
            "Regulation: Proposed real-time transaction reporting rule",
            "Agency: Federal Financial Regulatory Agency",
            "Timeline: 60-day comment period, 18-month implementation proposed",
            "Client concern: Current systems cannot support real-time reporting",
            "Estimated compliance cost: $5-8 million in system upgrades"
        ],
        "prompt": """Task: Draft an outline for a regulatory comment letter (bullet points acceptable).

Structure the outline with these sections:
1. Introduction (who is commenting)
2. Summary of concerns
3. Specific implementation challenges
4. Cost impact analysis needed
5. Request for extended timeline or phased implementation

Keep it factual and professional. This is an outline only, not a complete letter.

Include the required disclaimer at the end."""
    }

def case_international_contract():
    """International: Cross-border contract email to client"""
    return {
        "task_name": "international_contract_email",
        "facts": [
            "Client: US manufacturing company",
            "Situation: Negotiating supply agreement with European vendor",
            "Vendor proposal: Contract governed by vendor's country law, disputes in vendor's courts",
            "Client question: What are our options?",
            "Contract value: Approximately $2 million annually"
        ],
        "prompt": """Task: Draft a brief email to the client (300 words maximum).

Content to cover:
- Explain what 'governing law' and 'forum selection' clauses mean (in plain English)
- Why these clauses matter in international contracts
- General options available (without making specific recommendations)
- Note that specific advice requires jurisdiction analysis

Tone: Professional but accessible for non-lawyer business client.

Include the required disclaimer at the end."""
    }

def case_teaching_syllabus():
    """Teaching/Academia: Syllabus policy on student AI use"""
    return {
        "task_name": "teaching_ai_policy",
        "facts": [
            "Course: Legal Writing and Drafting (1L required course)",
            "Institution: Law school",
            "Instructor preference: Allow AI for research and outlining, prohibit for final drafts",
            "Concern: Students need to develop core drafting skills",
            "Academic integrity: Clear attribution rules needed"
        ],
        "prompt": """Task: Draft a syllabus section on AI tool usage (400 words maximum).

Content to include:
- Which assignments permit AI use and which do not
- Pedagogical rationale (why learning to draft without AI matters)
- Disclosure/attribution requirements when AI is used
- Academic integrity consequences for violations
- Tone: Clear, authoritative, but student-friendly

Include the required disclaimer at the end."""
    }

# Load all cases
MINI_CASES = [
    case_criminal_bail(),
    case_regulatory_comment(),
    case_international_contract(),
    case_teaching_syllabus()
]

print("=" * 60)
print("MINI-CASES LOADED - SIMPLIFIED VERSION")
print("=" * 60)
print("These cases use clearer, more structured prompts designed")
print("to elicit JSON-formatted responses from Claude.\n")

for i, case in enumerate(MINI_CASES, 1):
    print(f"{i}. {case['task_name']}")
    print(f"   Facts provided: {len(case['facts'])}")
    print(f"   Prompt length: {len(case['prompt'])} chars\n")

print(f"✅ All {len(MINI_CASES)} mini-cases ready to run")
print("=" * 60)

MINI-CASES LOADED - SIMPLIFIED VERSION
These cases use clearer, more structured prompts designed
to elicit JSON-formatted responses from Claude.

1. criminal_bail_letter
   Facts provided: 6
   Prompt length: 334 chars

2. regulatory_comment_letter
   Facts provided: 6
   Prompt length: 440 chars

3. international_contract_email
   Facts provided: 5
   Prompt length: 459 chars

4. teaching_ai_policy
   Facts provided: 5
   Prompt length: 406 chars

✅ All 4 mini-cases ready to run


##8.RUNNING THE FOUR DEMOS

###8.1.OVERVIEW

**Section 8: Run the Four Mini-Case Demos and Save Deliverables**

This section executes all four legal scenarios while implementing comprehensive error handling and documentation. Rather than assuming everything will work perfectly, it's designed to continue functioning even when individual cases encounter problems, creating a realistic model for professional practice where not everything always goes smoothly.

The execution loop processes each case individually within error-catching blocks. If one case fails, the notebook doesn't crash—it logs the error, creates an error deliverable for that case, and moves on to the next one. This resilience matters because you might run this notebook dozens of times while developing prompts or testing different fact patterns. Having the entire process halt because of one problematic case would make iteration frustrating and slow.

Progress indicators appear throughout execution. You see which case is being processed, whether the model call succeeded, and which files were saved. These status messages serve both technical and psychological purposes. Technically, they help diagnose problems when things go wrong. Psychologically, they provide reassurance that the notebook is working rather than frozen, especially important during the API calls which can take several seconds each.

Each successful case generates two deliverable files. The structured data file captures everything in machine-readable format—facts, assumptions, questions, risks, draft output, verification status. The human-readable text file presents the same information formatted for easy review, with clear section headers and the required disclaimer prominently displayed at the top. This dual format acknowledges that both humans and automated systems may need to process these outputs.

Even failed cases generate deliverables. The error structure includes the error message, empty lists for standard fields, and a high-severity risk noting the system failure. This ensures that your deliverables folder contains a file for every case attempted, making it easy to identify which cases succeeded and which need attention.

The statistics tracking counts successful versus failed cases throughout execution. At the end, a summary table displays case names, number of open questions identified, highest risk severity, and completion status using visual indicators. This table provides an at-a-glance assessment of the entire run—did everything work, or do certain cases need investigation?

The minimum standard document created here serves as educational material. It enumerates the five core controls that lawyers should implement when using artificial intelligence for drafting: minimize sensitive inputs, stick to facts-only tasks, verify all claims, maintain logs for material use, and require lawyer review before client-facing use. This document can be shared with colleagues, included in training materials, or referenced in firm policies.

From a governance perspective, this section demonstrates appropriate quality control. You're not blindly accepting model outputs—you're tracking what succeeded, what failed, what risks were identified, and creating documentation for human review. The summary statistics provide metrics for evaluating whether the system is working reliably enough for your purposes.

###8.2.CODE AND IMPLEMENTATION

In [17]:
# Cell 8: Run the 4 Mini-Case Demos + Save Deliverables

import traceback

print("=" * 60)
print("RUNNING MINI-CASE DEMOS")
print("=" * 60)
print(f"Total cases to process: {len(MINI_CASES)}\n")

results_summary = []
successful_cases = 0
failed_cases = 0

for idx, case in enumerate(MINI_CASES, 1):
    print(f"[{idx}/{len(MINI_CASES)}] Processing: {case['task_name']}")
    print("-" * 60)

    try:
        # Call Claude with error handling
        result = call_claude(case["task_name"], case["prompt"], case["facts"])

        # Check if there was an error in the result
        if "error" in result and result.get("draft_output") == "ERROR: Could not generate valid response":
            print(f"⚠️ Warning: {case['task_name']} completed with errors")
            failed_cases += 1
        else:
            print(f"✅ LLM call successful")
            successful_cases += 1

        # Save JSON deliverable
        json_path = DELIVERABLES_DIR / f"{case['task_name']}_output.json"
        write_json(json_path, result)
        print(f"✅ Saved JSON: {json_path.name}")

        # Create human-readable text deliverable
        txt_content = "=" * 60 + "\n"
        txt_content += f"TASK: {result.get('task', case['task_name'])}\n"
        txt_content += "=" * 60 + "\n\n"

        txt_content += "⚠️ DISCLAIMER: This is a draft only. Not legal advice.\n"
        txt_content += "Human lawyer review required before any use.\n\n"

        txt_content += "-" * 60 + "\n"
        txt_content += "FACTS PROVIDED:\n"
        txt_content += "-" * 60 + "\n"
        facts_list = result.get('facts_provided', [])
        if facts_list:
            for fact in facts_list:
                txt_content += f"• {fact}\n"
        else:
            txt_content += "(none)\n"

        txt_content += "\n" + "-" * 60 + "\n"
        txt_content += "ASSUMPTIONS MADE:\n"
        txt_content += "-" * 60 + "\n"
        assumptions = result.get('assumptions', [])
        if assumptions:
            for assumption in assumptions:
                txt_content += f"• {assumption}\n"
        else:
            txt_content += "(none)\n"

        txt_content += "\n" + "-" * 60 + "\n"
        txt_content += "OPEN QUESTIONS:\n"
        txt_content += "-" * 60 + "\n"
        open_questions = result.get('open_questions', [])
        if open_questions:
            for question in open_questions:
                txt_content += f"• {question}\n"
        else:
            txt_content += "(none)\n"

        txt_content += "\n" + "-" * 60 + "\n"
        txt_content += "DRAFT OUTPUT:\n"
        txt_content += "-" * 60 + "\n"
        txt_content += result.get('draft_output', 'N/A') + "\n"

        txt_content += "\n" + "-" * 60 + "\n"
        txt_content += f"VERIFICATION STATUS: {result.get('verification_status', 'Not verified')}\n"
        txt_content += "-" * 60 + "\n"
        questions_verify = result.get('questions_to_verify', [])
        if questions_verify:
            txt_content += "Questions to verify:\n"
            for question in questions_verify:
                txt_content += f"• {question}\n"
        else:
            txt_content += "(none)\n"

        txt_content += "\n" + "-" * 60 + "\n"
        txt_content += "IDENTIFIED RISKS:\n"
        txt_content += "-" * 60 + "\n"
        risks = result.get('risks', [])
        if risks:
            for risk in risks:
                risk_type = risk.get('type', 'unknown')
                severity = risk.get('severity', 'unknown')
                note = risk.get('note', 'No description')
                txt_content += f"[{severity.upper()}] {risk_type}: {note}\n"
        else:
            txt_content += "(none identified)\n"

        txt_content += "\n" + "=" * 60 + "\n"

        # Save text deliverable
        txt_path = DELIVERABLES_DIR / f"{case['task_name']}_draft.txt"
        with open(txt_path, 'w', encoding='utf-8') as f:
            f.write(txt_content)
        print(f"✅ Saved TXT: {txt_path.name}")

        # Collect summary info
        max_severity = "low"
        for risk in risks:
            risk_severity = risk.get('severity', 'low')
            if risk_severity == 'high':
                max_severity = 'high'
                break
            elif risk_severity == 'medium' and max_severity == 'low':
                max_severity = 'medium'

        results_summary.append({
            "case": case['task_name'],
            "open_questions": len(open_questions),
            "max_risk": max_severity,
            "status": "completed" if "error" not in result else "completed_with_errors"
        })

        print(f"✅ Case completed: {case['task_name']}\n")

    except Exception as e:
        print(f"❌ ERROR processing {case['task_name']}: {str(e)}")
        print("Stack trace:")
        traceback.print_exc()

        failed_cases += 1

        # Create error deliverable
        error_result = {
            "task": case['task_name'],
            "error": str(e),
            "facts_provided": case['facts'],
            "assumptions": [],
            "open_questions": ["Processing error occurred"],
            "risks": [{"type": "other", "severity": "high", "note": f"System error: {str(e)}"}],
            "draft_output": f"ERROR: Could not process this case. Error: {str(e)}",
            "verification_status": "Not verified",
            "questions_to_verify": []
        }

        # Save error result
        error_json_path = DELIVERABLES_DIR / f"{case['task_name']}_output.json"
        write_json(error_json_path, error_result)

        results_summary.append({
            "case": case['task_name'],
            "open_questions": 0,
            "max_risk": "high",
            "status": "failed"
        })

        print(f"⚠️ Error logged for: {case['task_name']}\n")

# Create minimum standard document
print("=" * 60)
print("Creating minimum standard document...")
print("=" * 60)

standard_content = """============================================================
MINIMUM STANDARD FOR SAFE AI USE AT LEVEL 1
============================================================

For lawyers using chatbot AI tools for drafting and client
communication, the following minimum controls are required:

1. MINIMIZE SENSITIVE INPUTS
   - Redact personally identifiable information
   - Anonymize client details where possible
   - Use only facts necessary for the task
   - Never paste raw client communications

2. FACTS-ONLY DRAFTING
   - Use AI only for drafting based on provided facts
   - Do NOT ask AI to conduct legal research
   - Do NOT rely on AI-generated citations or authorities
   - Treat all legal references as "Not verified"

3. VERIFY FACTUAL CLAIMS
   - Check every factual assertion in AI output
   - Verify dates, names, amounts, procedural history
   - Cross-reference against source documents
   - Flag unverified claims for investigation

4. MAINTAIN LOGS FOR MATERIAL USE
   - Document when AI was used substantively
   - Keep copies of prompts and outputs
   - Record review and edits made
   - Create audit trail for file retention

5. LAWYER REVIEW BEFORE CLIENT-FACING USE
   - Never send AI output directly to clients
   - Review for accuracy, tone, completeness
   - Ensure appropriate disclaimers
   - Apply professional judgment to every output

6. UNDERSTAND YOUR JURISDICTION'S RULES
   - Ethics opinions on AI use vary by state
   - Some jurisdictions require specific disclosures
   - Competence duty includes understanding AI tools
   - When in doubt, consult ethics counsel

============================================================
WHY THESE CONTROLS MATTER
============================================================

Level 1 chatbots are powerful tools but have critical
limitations:
- They can hallucinate facts, citations, and authorities
- They lack up-to-date legal research capabilities
- They cannot assess privilege or confidentiality
- They don't understand your specific case context

These controls bridge the gap between AI capability and
professional responsibility. As you move to more advanced
AI uses (Level 2: research, Level 3: autonomous agents),
additional controls become necessary.

Remember: The goal is not to avoid AI, but to use it
safely and effectively within ethical bounds.

============================================================
CAPABILITY INCREASES → RISK INCREASES → CONTROLS INCREASE
============================================================

Level 1 (Chatbots - This Chapter):
  Capability: Facts-only drafting, client communication
  Risks: Hallucination, missing facts, tone issues
  Controls: Redaction, fact separation, human review

Level 2 (Research Assistants):
  Capability: Legal research, cite-checking, case analysis
  Risks: Incorrect citations, outdated law, jurisdiction errors
  Controls: All Level 1 + citation verification, jurisdiction checks

Level 3 (Autonomous Agents):
  Capability: Multi-step workflows, document assembly, analysis
  Risks: All above + scope creep, unauthorized practice
  Controls: All Level 2 + strict scope limits, enhanced oversight

============================================================
"""

standard_content += f"\nGenerated: {now_iso()}\n"
standard_content += "Chapter 1 - Level 1 Chatbots with Governance Controls\n"
standard_content += "Author: Alejandro Reynoso, Chief Scientist DEFI CAPITAL RESEARCH\n"
standard_content += "============================================================\n"

standard_path = DELIVERABLES_DIR / "level1_minimum_standard.txt"
with open(standard_path, 'w', encoding='utf-8') as f:
    f.write(standard_content)

print(f"✅ Minimum standard saved: {standard_path.name}\n")

# Display summary table
print("=" * 60)
print("RESULTS SUMMARY")
print("=" * 60)
print(f"{'Case Name':<40} {'Open Q':>8} {'Risk':>10} {'Status':>15}")
print("-" * 60)

for item in results_summary:
    case_name = item['case'][:38]  # Truncate if too long
    open_q = item['open_questions']
    risk = item['max_risk']
    status = item['status']

    # Color coding with emoji
    if status == "completed":
        status_display = "✅ completed"
    elif status == "completed_with_errors":
        status_display = "⚠️ with errors"
    else:
        status_display = "❌ failed"

    print(f"{case_name:<40} {open_q:>8} {risk:>10} {status_display:>15}")

print("-" * 60)
print(f"Total: {len(MINI_CASES)} cases | "
      f"Successful: {successful_cases} | "
      f"Failed: {failed_cases}")
print("=" * 60)

# Final statistics
print("\n" + "=" * 60)
print("GOVERNANCE ARTIFACTS SUMMARY")
print("=" * 60)

# Count files in deliverables
deliverable_files = list(DELIVERABLES_DIR.glob("*"))
print(f"Deliverable files created: {len(deliverable_files)}")

# Check risk log
with open(risk_log_path, 'r') as f:
    current_risk_log = json.load(f)
total_risks = len(current_risk_log.get('risks', []))
print(f"Total risks logged: {total_risks}")

# Check prompts log
with open(prompts_log_path, 'r') as f:
    prompt_lines = f.readlines()
print(f"Total API calls logged: {len(prompt_lines)}")

print("\n📁 All deliverables location: " + str(DELIVERABLES_DIR))

if failed_cases > 0:
    print(f"\n⚠️ WARNING: {failed_cases} case(s) encountered errors.")
    print("   Check the individual output files for error details.")
    print("   Common causes: API rate limits, JSON parsing issues, network errors.")
else:
    print("\n✅ All mini-cases completed successfully!")

print("\n" + "=" * 60)
print("NEXT STEPS")
print("=" * 60)
print("1. Review the deliverable files in the deliverables/ folder")
print("2. Examine the risk_log.json for identified risks")
print("3. Check prompts_log.jsonl for API call history")
print("4. Proceed to Cell 9 for your own exercise")
print("5. Run Cell 10 to create final audit bundle")
print("=" * 60)

RUNNING MINI-CASE DEMOS
Total cases to process: 4

[1/4] Processing: criminal_bail_letter
------------------------------------------------------------
  ✅ Valid JSON extracted (attempt 1)
✅ LLM call successful
✅ Saved JSON: criminal_bail_letter_output.json
✅ Saved TXT: criminal_bail_letter_draft.txt
✅ Case completed: criminal_bail_letter

[2/4] Processing: regulatory_comment_letter
------------------------------------------------------------
  ⚠️ JSON extraction failed (attempt 1). Retrying with stricter prompt...
  ⚠️ JSON extraction failed (attempt 2). Retrying with stricter prompt...
  ❌ Could not extract JSON after 3 attempts
  ❌ Could not get valid JSON from Claude for regulatory_comment_letter
✅ LLM call successful
✅ Saved JSON: regulatory_comment_letter_output.json
✅ Saved TXT: regulatory_comment_letter_draft.txt
✅ Case completed: regulatory_comment_letter

[3/4] Processing: international_contract_email
------------------------------------------------------------
  ✅ Valid JSON 

##9.USER EXERCISE

###9.1.OVERVIEW

**Section 8: Run the Four Mini-Case Demos and Save Deliverables**

This section executes all four legal scenarios while implementing comprehensive error handling and documentation. Rather than assuming everything will work perfectly, it's designed to continue functioning even when individual cases encounter problems, creating a realistic model for professional practice where not everything always goes smoothly.

The execution loop processes each case individually within error-catching blocks. If one case fails, the notebook doesn't crash—it logs the error, creates an error deliverable for that case, and moves on to the next one. This resilience matters because you might run this notebook dozens of times while developing prompts or testing different fact patterns. Having the entire process halt because of one problematic case would make iteration frustrating and slow.

Progress indicators appear throughout execution. You see which case is being processed, whether the model call succeeded, and which files were saved. These status messages serve both technical and psychological purposes. Technically, they help diagnose problems when things go wrong. Psychologically, they provide reassurance that the notebook is working rather than frozen, especially important during the API calls which can take several seconds each.

Each successful case generates two deliverable files. The structured data file captures everything in machine-readable format—facts, assumptions, questions, risks, draft output, verification status. The human-readable text file presents the same information formatted for easy review, with clear section headers and the required disclaimer prominently displayed at the top. This dual format acknowledges that both humans and automated systems may need to process these outputs.

Even failed cases generate deliverables. The error structure includes the error message, empty lists for standard fields, and a high-severity risk noting the system failure. This ensures that your deliverables folder contains a file for every case attempted, making it easy to identify which cases succeeded and which need attention.

The statistics tracking counts successful versus failed cases throughout execution. At the end, a summary table displays case names, number of open questions identified, highest risk severity, and completion status using visual indicators. This table provides an at-a-glance assessment of the entire run—did everything work, or do certain cases need investigation?

The minimum standard document created here serves as educational material. It enumerates the five core controls that lawyers should implement when using artificial intelligence for drafting: minimize sensitive inputs, stick to facts-only tasks, verify all claims, maintain logs for material use, and require lawyer review before client-facing use. This document can be shared with colleagues, included in training materials, or referenced in firm policies.

From a governance perspective, this section demonstrates appropriate quality control. You're not blindly accepting model outputs—you're tracking what succeeded, what failed, what risks were identified, and creating documentation for human review. The summary statistics provide metrics for evaluating whether the system is working reliably enough for your purposes.

###9.2.CODE AND IMPLEMENTATION

In [18]:
# Cell 9: User Exercise: Your Turn (Safe Intake + Draft)

print("=" * 60)
print("YOUR TURN: SAFE AI DRAFTING EXERCISE")
print("=" * 60)
print("\nThis exercise will walk you through:")
print("1. Entering a scenario (with automatic redaction)")
print("2. Reviewing what was redacted")
print("3. Choosing draft type (email or memo)")
print("4. Generating a governed draft\n")
print("⚠️ Remember: Use FAKE or ANONYMIZED data only for practice\n")

# Get user scenario
print("STEP 1: Enter your scenario")
print("-" * 60)
scenario = input("Paste your scenario (or type a brief description):\n\n")

if not scenario.strip():
    print("\n⚠️ No scenario provided. Using default example...")
    scenario = "Client needs email explaining delay in filing due to missing documents from opposing party."

# Redact
print("\n" + "=" * 60)
print("STEP 2: Automatic Redaction")
print("=" * 60)
redacted_scenario, removed_items = redact(scenario)

if removed_items:
    print("\n⚠️ REDACTED ITEMS:")
    for item in removed_items:
        print(f"  • {item}")
    print(f"\nOriginal length: {len(scenario)} chars")
    print(f"Redacted length: {len(redacted_scenario)} chars")
else:
    print("\n✅ No sensitive patterns detected (but always double-check manually)")

print("\nREDACTED SCENARIO:")
print("-" * 60)
print(redacted_scenario)
print("-" * 60)

# Confirm
confirm = input("\nProceed with this redacted version? (yes/no): ").strip().lower()
if confirm != 'yes':
    print("\n⚠️ Exercise cancelled. Please re-run cell to try again.")
else:
    # Choose output type
    print("\n" + "=" * 60)
    print("STEP 3: Choose Draft Type")
    print("=" * 60)
    print("1 = Client email")
    print("2 = Short memo")
    choice = input("Enter 1 or 2: ").strip()

    if choice == "1":
        output_type = "email"
        prompt = dedent("""
        Draft a professional client email based on the scenario provided.
        Keep it concise, clear, and appropriate in tone.
        Include next steps or action items if relevant.
        """).strip()
    else:
        output_type = "memo"
        prompt = dedent("""
        Draft a short internal memo (2-3 paragraphs) based on the scenario provided.
        Summarize the situation and recommend next steps.
        """).strip()

    # Generate draft
    print("\n" + "=" * 60)
    print("STEP 4: Generating Draft...")
    print("=" * 60)

    facts = [redacted_scenario]
    result = call_claude(f"user_exercise_{output_type}", prompt, facts)

    # Save deliverables
    json_path = DELIVERABLES_DIR / f"user_exercise_output.json"
    write_json(json_path, result)

    txt_content = dedent(f"""
    ============================================================
    USER EXERCISE: {output_type.upper()}
    ============================================================

    ⚠️ DISCLAIMER: This is a draft only. Not legal advice.
    Human lawyer review required before any use.

    ------------------------------------------------------------
    REDACTION SUMMARY:
    ------------------------------------------------------------
    {chr(10).join('• ' + item for item in removed_items) if removed_items else '• No items redacted'}

    ------------------------------------------------------------
    FACTS PROVIDED:
    ------------------------------------------------------------
    {redacted_scenario}

    ------------------------------------------------------------
    OPEN QUESTIONS:
    ------------------------------------------------------------
    {chr(10).join('• ' + q for q in result.get('open_questions', [])) if result.get('open_questions') else '(none)'}

    ------------------------------------------------------------
    DRAFT OUTPUT:
    ------------------------------------------------------------
    {result.get('draft_output', 'N/A')}

    ------------------------------------------------------------
    VERIFICATION STATUS: {result.get('verification_status', 'Not verified')}
    ------------------------------------------------------------

    ------------------------------------------------------------
    IDENTIFIED RISKS:
    ------------------------------------------------------------
    """).strip()

    for risk in result.get('risks', []):
        txt_content += f"\n[{risk['severity'].upper()}] {risk['type']}: {risk['note']}"

    txt_content += "\n\n============================================================\n"

    txt_path = DELIVERABLES_DIR / f"user_exercise_draft.txt"
    with open(txt_path, 'w') as f:
        f.write(txt_content)

    print(f"\n✅ Your {output_type} draft is ready!")
    print(f"\nSaved to:")
    print(f"  • {json_path}")
    print(f"  • {txt_path}")

    print("\n" + "=" * 60)
    print("REDACTION SUMMARY")
    print("=" * 60)
    if removed_items:
        for item in removed_items:
            print(f"  • {item}")
    else:
        print("  • No sensitive patterns detected")

    print("\n⚠️ NEXT STEPS:")
    print("  1. Review the draft carefully")
    print("  2. Verify all factual claims")
    print("  3. Check tone and completeness")
    print("  4. Address open questions")
    print("  5. Apply your professional judgment")

YOUR TURN: SAFE AI DRAFTING EXERCISE

This exercise will walk you through:
1. Entering a scenario (with automatic redaction)
2. Reviewing what was redacted
3. Choosing draft type (email or memo)
4. Generating a governed draft

⚠️ Remember: Use FAKE or ANONYMIZED data only for practice

STEP 1: Enter your scenario
------------------------------------------------------------
Paste your scenario (or type a brief description):

SUING A OLI COMPANY OIL SPILL

STEP 2: Automatic Redaction

✅ No sensitive patterns detected (but always double-check manually)

REDACTED SCENARIO:
------------------------------------------------------------
SUING A OLI COMPANY OIL SPILL
------------------------------------------------------------

Proceed with this redacted version? (yes/no): YES

STEP 3: Choose Draft Type
1 = Client email
2 = Short memo
Enter 1 or 2: 2

STEP 4: Generating Draft...
  ✅ Valid JSON extracted (attempt 1)

✅ Your memo draft is ready!

Saved to:
  • /content/ai_law_ch1_runs/run_2026010

##10.BUNDLE ARTIFACTS AND AUDIT README

###10.1.OVERVIEW

**Section 10: Bundle Artifacts, Create Audit Readme, and Generate Zip**

This section packages everything created during the session into a single downloadable archive with comprehensive documentation. Rather than scattering files across multiple locations or expecting users to manually collect outputs, it creates a complete audit bundle that meets professional responsibility standards for record-keeping.

The audit readme serves as the guidebook for the entire bundle. It explains what each file contains, why it exists, and how to interpret it. This documentation matters because six months from now, you or a colleague might need to understand what happened in this session. The readme provides that context without requiring you to remember technical details or reconstruct the process from memory.

The readme explains how to review the prompts log safely. Since the log contains redacted versions of inputs and outputs plus cryptographic fingerprints, reviewers can verify what happened without exposing full potentially-confidential content. The fingerprints allow integrity verification—you can prove the logged content hasn't been altered—while the redacted previews provide enough context to understand what each interaction involved.

Reproducibility instructions tell readers exactly what software environment, model version, parameters, and process were used. If you need to recreate these results, perhaps because a client questions an output or you want to validate the approach with different facts, these instructions provide the roadmap. The readme acknowledges that exact reproduction may not be possible due to model updates or non-deterministic behavior, but documents everything that can be controlled.

Professional responsibility notes explain why this documentation matters for legal ethics. The bundle serves as evidence of competence in tool use, reasonable supervision of artificial intelligence-assisted work, and compliance with file retention requirements. It demonstrates quality control processes rather than blind reliance on technology. These explanations help lawyers understand that governance isn't bureaucratic overhead—it's how you demonstrate professional responsibility when using new tools.

The file inventory lists every artifact created, showing file paths and sizes. This manifest ensures nothing gets lost and provides a quick overview of bundle contents. The summary statistics—total files, total size—give a sense of the documentation scope.

The contents checklist verifies that all required artifacts are present: manifest with environment details, complete prompt log, risk tracking log, all mini-case deliverables, minimum standard document, environment snapshot, and the readme itself. Visual indicators show at a glance whether the bundle is complete or missing components.

Download instructions provide multiple methods for retrieving the zip file, recognizing that users have different comfort levels with technical interfaces. Some prefer clicking through the file browser, others prefer running code commands. Offering both options makes the tool accessible to lawyers with varying technical backgrounds.

From a governance perspective, this final section completes the audit trail loop. You started by documenting what you intended to do, logged what actually happened, and now package everything for retention and future reference. This closed loop provides defensibility if your artificial intelligence usage is ever questioned or audited.

###10.2.CODE AND IMPLEMENTATION

In [19]:
# Cell 10: Bundle Artifacts + Audit Readme + Zip

print("=" * 60)
print("CREATING AUDIT BUNDLE")
print("=" * 60)

# Create audit README
audit_readme = dedent(f"""
============================================================
AUDIT README - AI FOR LAWYERS CHAPTER 1
============================================================

Run ID: {manifest['run_id']}
Generated: {now_iso()}
Author: Alejandro Reynoso
Model: {MODEL}

This bundle contains a complete audit trail of an AI-assisted
legal drafting session using Level 1 governance controls.

============================================================
CONTENTS OF THIS BUNDLE
============================================================

1. run_manifest.json
   - Unique run identifier
   - Timestamp and environment details
   - Model and parameters used
   - Python dependencies (pip_freeze.txt)

2. prompts_log.jsonl
   - Every prompt sent to the AI model
   - Every response received
   - All content is redacted for confidentiality
   - Includes SHA-256 hashes for integrity verification
   - JSONL format (one JSON object per line)

3. risk_log.json
   - All identified risks across all tasks
   - Risk types: confidentiality, privilege, hallucination,
     missing_facts, unauthorized_practice, tone, other
   - Severity levels: low, medium, high
   - Timestamped entries for each risk

4. deliverables/ folder
   - Output files for each demo mini-case
   - JSON format (machine-readable)
   - TXT format (human-readable)
   - User exercise outputs
   - Minimum standard document

5. pip_freeze.txt
   - Complete list of Python packages and versions
   - Enables reproducibility of the environment

6. AUDIT_README.txt (this file)

============================================================
HOW TO REVIEW THE PROMPT LOG SAFELY
============================================================

The prompts_log.jsonl file contains redacted versions of
all prompts and responses. To review:

1. Open the file in a text editor that supports JSONL
2. Each line is a separate JSON object
3. Key fields:
   - timestamp: When the call was made
   - task_name: Which mini-case or exercise
   - prompt_hash: SHA-256 hash of full prompt
   - prompt_redacted: First 500 chars of redacted prompt
   - response_hash: SHA-256 hash of full response
   - response_redacted: First 500 chars of response

4. Hashes allow verification without exposing full content
5. If you need to audit full content, check original source
   (but remember: original may contain unredacted data)

============================================================
HOW TO REPRODUCE THIS RUN
============================================================

To reproduce these results:

1. Environment:
   - Python version: {manifest['environment']['python_version']}
   - Platform: {manifest['environment']['platform']}
   - Dependencies: See pip_freeze.txt

2. Model:
   - Name: {MODEL}
   - Temperature: 0.2
   - Max tokens: 1200

3. Process:
   - Run the notebook cells 1-10 in sequence
   - Use same prompts from prompts_log.jsonl
   - Apply same redaction rules (see Cell 5)

Note: Exact reproduction may not be possible due to:
- Model updates/versioning
- Non-deterministic elements (even at low temperature)
- Differences in input data redaction

============================================================
PROFESSIONAL RESPONSIBILITY NOTES
============================================================

This bundle serves as documentation for:
- Competence in AI tool use (ethical duty to understand)
- Reasonable supervision of AI-assisted work
- File retention requirements
- Quality control and review process

Recommended retention:
- Keep with client file if used in matter
- Minimum retention period per your jurisdiction's rules
- Consider as part of work product documentation

============================================================
QUESTIONS OR CONCERNS
============================================================

If you have questions about:
- The governance framework: Review Cell 1 (orientation)
- Specific risks identified: See risk_log.json
- Reproduction: See manifest and pip_freeze.txt
- Ethics compliance: Consult your jurisdiction's rules

Remember: AI is a tool. Professional judgment is required.

============================================================
DISCLAIMER
============================================================

All outputs in this bundle are drafts only.
NOT LEGAL ADVICE.
Human lawyer review required before any use.

============================================================
""").strip()

readme_path = RUN_DIR / "AUDIT_README.txt"
with open(readme_path, 'w') as f:
    f.write(audit_readme)

print(f"✅ Audit README created: {readme_path.name}")

# List all files
print("\n" + "=" * 60)
print("FILE INVENTORY")
print("=" * 60)

all_files = []
for item in RUN_DIR.rglob('*'):
    if item.is_file():
        rel_path = item.relative_to(RUN_DIR)
        size = item.stat().st_size
        all_files.append((str(rel_path), size))

all_files.sort()

for filepath, size in all_files:
    size_kb = size / 1024
    print(f"  {filepath:<50} {size_kb:>8.1f} KB")

print(f"\nTotal files: {len(all_files)}")

# Create zip bundle
import shutil

zip_base = f"/content/ai_law_ch1_run_{timestamp}"
zip_path = shutil.make_archive(zip_base, 'zip', RUN_DIR)

print("\n" + "=" * 60)
print("ZIP BUNDLE CREATED")
print("=" * 60)
print(f"Location: {zip_path}")
print(f"Size: {Path(zip_path).stat().st_size / 1024 / 1024:.2f} MB")

print("\n" + "=" * 60)
print("BUNDLE CONTENTS CHECKLIST")
print("=" * 60)
checklist = [
    ("Run manifest with environment details", "run_manifest.json" in str(all_files)),
    ("Complete prompt log (JSONL)", "prompts_log.jsonl" in str(all_files)),
    ("Risk tracking log", "risk_log.json" in str(all_files)),
    ("All mini-case deliverables", len([f for f, _ in all_files if 'deliverables' in f]) >= 9),
    ("Minimum standard document", "level1_minimum_standard.txt" in str(all_files)),
    ("Python environment snapshot", "pip_freeze.txt" in str(all_files)),
    ("Audit README", "AUDIT_README.txt" in str(all_files)),
]

for item, present in checklist:
    status = "✅" if present else "❌"
    print(f"{status} {item}")

print("\n" + "=" * 60)
print("DOWNLOAD INSTRUCTIONS")
print("=" * 60)
print("1. Click the folder icon in the left sidebar")
print("2. Navigate to /content/")
print(f"3. Find: ai_law_ch1_run_{timestamp}.zip")
print("4. Right-click → Download")
print("\nOR run this in a new cell:")
print(f"from google.colab import files")
print(f"files.download('{zip_path}')")

print("\n" + "=" * 60)
print("SESSION COMPLETE")
print("=" * 60)
print("\n✅ All governance artifacts generated")
print("✅ All mini-cases completed with risk tracking")
print("✅ User exercise recorded and bundled")
print("✅ Complete audit trail ready for download")
print("\n📚 Chapter 1 Learning Objectives Achieved:")
print("  • Structured AI prompts with fact/assumption separation")
print("  • Governance artifacts for professional responsibility")
print("  • Risk identification and flagging")
print("  • Understanding Level 1 limitations and controls")
print("  • Minimum standards for safe AI use in legal practice")
print("\n⚠️ REMEMBER: Capability ⇒ Risk ⇒ Controls")
print("   Always apply professional judgment.\n")

CREATING AUDIT BUNDLE
✅ Audit README created: AUDIT_README.txt

FILE INVENTORY
  AUDIT_README.txt                                        4.3 KB
  deliverables/criminal_bail_letter_draft.txt             4.2 KB
  deliverables/criminal_bail_letter_output.json           3.5 KB
  deliverables/international_contract_email_draft.txt      4.7 KB
  deliverables/international_contract_email_output.json      4.1 KB
  deliverables/level1_minimum_standard.txt                3.4 KB
  deliverables/regulatory_comment_letter_draft.txt        1.7 KB
  deliverables/regulatory_comment_letter_output.json      0.9 KB
  deliverables/teaching_ai_policy_draft.txt               5.0 KB
  deliverables/teaching_ai_policy_output.json             4.4 KB
  deliverables/user_exercise_draft.txt                    4.4 KB
  deliverables/user_exercise_output.json                  4.2 KB
  pip_freeze.txt                                         12.4 KB
  prompts_log.jsonl                                       7.6 KB
  risk_

##11.COONCLUSIONS